In [1]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns

import cufflinks as cf
import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objs as go
import plotly as py
cf.go_offline()

from scipy import stats
from scipy.stats import skew
import scipy.signal as signal




# Modelling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import roc_curve
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [2]:
df = pd.read_csv('Data_for_UCI_named.csv')

In [3]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [4]:
df.size

140000

In [5]:
df.shape

(10000, 14)

In [6]:
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

##### Map the Stabs, define the class

In [7]:
# Map 'stabf' values to numerical values
df['stabf'] = df['stabf'].map({'stable': 0, 'unstable': 1})

In [8]:
# Split the dataset into features (X) and the target variable (y)
X = df.drop(columns=['stab', 'stabf'])
y = df['stabf']


##### Split the Train-Test Data

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [10]:
df.columns

Index(['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2',
       'g3', 'g4', 'stab', 'stabf'],
      dtype='object')

##### Identify the categorical columns in X_train

In [11]:
categorical_columns = ['tau1', 'tau2', 'tau3', 'tau4']

##### Create a OneHotEncoder object

In [12]:
encoder = OneHotEncoder(handle_unknown='ignore')

In [13]:
# Fit the encoder on the training data
encoder.fit(X_train[categorical_columns])

OneHotEncoder(handle_unknown='ignore')

##### Random Forest

In [14]:
model1 = RandomForestClassifier()
model1.fit(X_train, y_train)

RandomForestClassifier()

In [15]:
y_pred1 = model1.predict(X_test)

In [16]:
accuracy1 = accuracy_score(y_test, y_pred1)

In [17]:
print("Accuracy on the test set:", round(accuracy1, 4))

Accuracy on the test set: 0.9235


##### XGBClassifier

In [18]:
model2 = XGBClassifier()
model2.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [19]:
y_pred2 = model2.predict(X_test)

In [20]:
accuracy2 = accuracy_score(y_test, y_pred2)

In [21]:
print("Accuracy on XGBClassifier:", round(accuracy2, 4))

Accuracy on XGBClassifier: 0.9455


##### LGBMClassifier

In [22]:
model3 = LGBMClassifier()
model3.fit(X_train, y_train)

LGBMClassifier()

In [23]:
y_pred3 = model3.predict(X_test)

In [24]:
accuracy3 = accuracy_score(y_test, y_pred3)

In [25]:
print("Accuracy on LGBMClassifier:", round(accuracy3, 4))

Accuracy on LGBMClassifier: 0.939


##### ExtraTree Classifier

In [26]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [27]:
model4 = ExtraTreesClassifier(random_state=1)

In [28]:
random_search = RandomizedSearchCV(estimator=model4, param_distributions=param_grid, 
                                   cv=5, n_iter=10, scoring='accuracy', n_jobs=-1, 
                                   verbose=1, random_state=1)
random_search.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=ExtraTreesClassifier(random_state=1),
                   n_jobs=-1,
                   param_distributions={'max_depth': [None, 5, 10],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 3],
                                        'min_samples_split': [2, 4, 6],
                                        'n_estimators': [100, 200, 300]},
                   random_state=1, scoring='accuracy', verbose=1)

In [29]:
best_params = random_search.best_params_
best_score = random_search.best_score_

In [30]:
print("Best Hyperparameters:", best_params)
print("Best Score:", best_score)

Best Hyperparameters: {'n_estimators': 200, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None}
Best Score: 0.9295


In [31]:
model4 = ExtraTreesClassifier(n_estimators=100, max_depth=None, min_samples_split=2, 
                                     min_samples_leaf=1, max_features='auto', random_state=1)

In [32]:
model4.fit(X_train, y_train)

ExtraTreesClassifier(random_state=1)

In [33]:
y_pred4 = model4.predict(X_test)

In [34]:
accuracy = accuracy_score(y_pred4, y_test)

In [35]:
print("Accuracy of ExtraTreeClassifier:", accuracy)

Accuracy of ExtraTreeClassifier: 0.928


In [36]:

#numeric_features = X.columns.tolist()
#categorical_features = []

In [37]:
# Create the transformer for feature scaling
#preprocessor = ColumnTransformer(
#    transformers=[
#        ('num', StandardScaler(), numeric_features)
 #   ])

In [38]:

# Fit and transform the data
#processed_df = preprocessor.fit_transform(X)


##### Length of Country Name

In [39]:
# Inspect the keys and transformers in the preprocessor
#for key, transformer in preprocessor.named_transformers_.items():
 #   print(key, transformer)


In [40]:
#X_processed_df = pd.DataFrame(processed_df, columns=numeric_features + preprocessor.named_transformers_['categorical'].get_feature_names_out(categorical_features))


In [41]:
# Create interaction features
#X_processed_df = pd.DataFrame(processed_df, columns=numeric_features + preprocessor.named_transformers_['cat'].get_feature_names(categorical_features))

In [42]:
#interaction_features = ['tau1', 'tau2', 'tau3', 'tau4']
#for i in range(len(interaction_features)):
#    for j in range(i+1, len(interaction_features)):
 #       feature_name = interaction_features[i] + '_x_' + interaction_features[j]
  #      X_processed_df[feature_name] = X_processed_df[interaction_features[i]] * X_processed_df[interaction_features[j]]

In [43]:
# Create polynomial features
#polynomial_features = ['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2', 'g3', 'g4']
#for feature in polynomial_features:
#    feature_name = feature + '_squared'
 #   X_processed_df[feature_name] = X_processed_df[feature] ** 2

In [44]:
# Display the updated dataset with new features
#print(X_processed_df.head())

##### Alpha Code Concatenation

In [45]:
# Split the data into training and test sets
#X_train, X_test, y_train, y_test = train_test_split(X_processed_df, y, test_size=0.2, random_state=1)

In [46]:
# Create and train the random forest classifier
#model1 = RandomForestClassifier(random_state=1)
#model1.fit(X_train, y_train)

In [47]:
# Create and train the random forest classifier
#model1 = RandomForestClassifier(random_state=1)
#model1.fit(X_train, y_train)

In [48]:
# Get feature importances
#feature_importances = pd.DataFrame({'Feature': X_processed_df.columns, 'Importance': rf_classifier.feature_importances_})
#feature_importances.sort_values(by='Importance', ascending=False, inplace=True)

In [49]:
# Display feature importances
#print(feature_importances)